# Subclassing API - Tensorflow 2.0

In [1]:
import tensorflow as tf
import numpy as np

## Be sure to used Tensorflow 2.0

In [2]:
assert hasattr(tf, "function") # Be sure to use tensorflow 2.0

### Import the dataset

In [3]:
from sklearn.preprocessing import StandardScaler
# Fashio MNIST
fashion_mnist = tf.keras.datasets.fashion_mnist
(images, targets), (_, _) = fashion_mnist.load_data()
# Get only a subpart of the dataset
images = images[:10000]
targets = targets [:10000]
# Reshape the dataset and convert to float
images = images.reshape(-1, 784)
images = images.astype(float)
# Normalize images
scaler = StandardScaler()
images = scaler.fit_transform(images)

## Create a model using the SubClassing API
The following call is not working because self.output is already set by the parent: tf.keras.Model

In [4]:
class CustomModel(tf.keras.Model):
    
    def __init__(self):
        super(CustomModel, self).__init__()

        # First in the init method you need to instanciate the layers you will use
        self.first_layer = tf.keras.layers.Dense(64)
        # WARNING: DO NOT CALL ONE OF YOUR CLASS VARIABLE: output
        self.output = tf.keras.layers.Dense(10, activation='softmax', name="d1")

    def call(self, x):
        # Then in the call method you can execute your operations
        layer1_out = self.first_layer(x)
        output = self.output_layer(layer1_out)
        return output

try:
    model = CustomModel()
except Exception as e:
    print("e=", e)

e= Can't set the attribute "output", likely because it conflicts with an existing read-only @property of the object. Please choose a different name.


In [6]:
class CustomModel(tf.keras.Model):
    
    def __init__(self):
        super(CustomModel, self).__init__()        
        # First in the init method you need to instanciate the layers you will use
        self.first_layer = tf.keras.layers.Dense(64, activation="relu", name="first_layer")
        self.output_layer = tf.keras.layers.Dense(10, activation='softmax', name="output_layer")

    def call(self, x):
        # Then in the call method you can execute your operations
        prev = self.first_layer(x)
        out = self.output_layer(prev)
        return out

model = CustomModel()
model.predict(images[0:1])

array([[0.00580786, 0.05870314, 0.65907234, 0.01117061, 0.06425457,
        0.02247835, 0.09529579, 0.03126691, 0.01721727, 0.03473315]],
      dtype=float32)

The cell above is now working: We can call model.predict as we would do with a basic keras model. <br> 
The <b>predict</b> method of the parent class called the <b>call</b> method from the <b>CustomModel</b> class

In [7]:
model.predict(images[0:1])

array([[0.00580786, 0.05870314, 0.65907234, 0.01117061, 0.06425457,
        0.02247835, 0.09529579, 0.03126691, 0.01721727, 0.03473315]],
      dtype=float32)

### Train the model

The model can be used as a normal keras model. Thus, to train it you just need to compile and to fit the model.

In [8]:
# Compile the model
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

In [9]:
history = model.fit(images, targets, epochs=10)

Epoch 1/10
313/313 [==============================] - 1s 2ms/step - loss: 1.1459 - accuracy: 0.6235
Epoch 2/10
313/313 [==============================] - 1s 2ms/step - loss: 0.5517 - accuracy: 0.8061
Epoch 3/10
313/313 [==============================] - 1s 2ms/step - loss: 0.4574 - accuracy: 0.8380
Epoch 4/10
313/313 [==============================] - 1s 2ms/step - loss: 0.4268 - accuracy: 0.8461
Epoch 5/10
313/313 [==============================] - 1s 2ms/step - loss: 0.3861 - accuracy: 0.8605
Epoch 6/10
313/313 [==============================] - 1s 2ms/step - loss: 0.3687 - accuracy: 0.8757
Epoch 7/10
313/313 [==============================] - 1s 2ms/step - loss: 0.3478 - accuracy: 0.8762
Epoch 8/10
313/313 [==============================] - 1s 2ms/step - loss: 0.3410 - accuracy: 0.8790
Epoch 9/10
313/313 [==============================] - 1s 2ms/step - loss: 0.3183 - accuracy: 0.8903
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 0.3206 - accuracy: 0.8884